In [1]:
import sys
sys.path.insert(0, "./../")

import os
import utils_modelling as utils

In [2]:
path_dataset = "./../../../dataset_used_split"
path_train = os.path.join(path_dataset, "train")
path_val = os.path.join(path_dataset, "val")
path_test = os.path.join(path_dataset, "test")

In [3]:
train_img, train_mask = utils.load_img_mask(path_train)
val_img, val_mask = utils.load_img_mask(path_val)
test_img, test_mask = utils.load_img_mask(path_test)

In [4]:
trainset = utils.create_dataset(train_img, train_mask)
valset = utils.create_dataset(val_img, val_mask)
testset = utils.create_dataset(test_img, test_mask)

In [5]:
unet_custom = utils.custom_unet()
unet_custom, ev_loss, ev_acc = utils.train_model(unet_custom, trainset, valset, testset, "unet_model")
print(f"test loss: {ev_loss}", f"test accuracy: {ev_acc}", sep="\n")

Epoch 1/10
16/16 [==============================] - 30s 717ms/step - loss: 0.7098 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.2834 - val_sparse_categorical_accuracy: 0.9825
Epoch 2/10
16/16 [==============================] - 7s 443ms/step - loss: 0.1452 - sparse_categorical_accuracy: 0.9826 - val_loss: 0.0763 - val_sparse_categorical_accuracy: 0.9825
Epoch 3/10
16/16 [==============================] - 8s 447ms/step - loss: 0.0740 - sparse_categorical_accuracy: 0.9826 - val_loss: 0.0644 - val_sparse_categorical_accuracy: 0.9825
Epoch 4/10
16/16 [==============================] - 6s 339ms/step - loss: 0.0667 - sparse_categorical_accuracy: 0.9826 - val_loss: 0.0683 - val_sparse_categorical_accuracy: 0.9825
Epoch 5/10
16/16 [==============================] - 5s 292ms/step - loss: 0.0729 - sparse_categorical_accuracy: 0.9826 - val_loss: 0.0596 - val_sparse_categorical_accuracy: 0.9825
Epoch 6/10
16/16 [==============================] - 5s 285ms/step - loss: 0.0673 - sparse_categoric